## Objective

Part pengerjaan ini bertujuan membangun sistem chatbot pajak profesional berbasis RAG (Retrieval-Augmented Generation) yang mengintegrasikan data FAQ.csv dari Direktorat Jenderal Pajak Indonesia dengan teknologi AI, untuk memberikan jawaban akurat terkait peraturan perpajakan dalam bahasa Indonesia. Sistem ini dirancang untuk: Memfilter pertanyaan non-pajak secara ketat menggunakan daftar keyword spesifik seperti PPh, PPN, NPWP; Mengambil informasi relevan dari database MongoDB dengan similarity search menggunakan embedding OpenAI; Menghasilkan respons terstruktur berformat profesional.

## Pengerjaan

## Import Libraries

In [2]:
!pip install langchain_mongodb

In [3]:
from pymongo import MongoClient
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os
import re
import pymongo
from langchain.prompts import PromptTemplate

In [4]:
# Load Environment Variables
load_dotenv(override=True)
MONGODB_URI = os.getenv("MONGO_URI")
OPENAI_KEY = os.getenv("OPENAI_API_KEY")

insight: Membaca kredensial dari file .env

In [5]:
# Initialize Embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=OPENAI_KEY,
    dimensions=1536
)

insight: Mengonversi teks FAQ menjadi vektor numerik; Model embedding khusus untuk teks pendek (ukuran 1536 dimensi)

In [6]:
# MongoDB Connection
client = MongoClient(MONGODB_URI)
collection = client['Astrax_db']['Astrax']

insight: Menghubungkan ke koleksi Astrax di database Astrax_db; Menggunakan index vector_index untuk pencarian vektor.

In [7]:
# Vector Store Configuration
vector_store = MongoDBAtlasVectorSearch(
    collection=collection,
    embedding=embeddings,
    index_name='vector_index',
    text_key="text"
)

insight: Menyimpan dan mencari dokumen FAQ dalam format vektor; set Kolom text digunakan sebagai konten utama.

In [8]:
# Model Configuration
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    openai_api_key=OPENAI_KEY,
    temperature=0
)

insight:
Menggunakan Model GPT-3.5-turbo untuk generasi jawaban; SetTemperature 0 untuk konsistensi jawaban.

In [9]:
template="""
Anda adalah asisten pajak digital Direktorat Jenderal Pajak (DJP) Indonesia yang bertugas membantu wajib pajak memahami informasi terkait pajak.

Berikut ini adalah kumpulan informasi yang relevan dari basis data DJP yang dapat digunakan untuk menjawab pertanyaan:

{context}

**Instruksi Penting:**
- Jawablah hanya berdasarkan informasi yang tersedia dalam konteks di atas.
- Jangan menambahkan informasi baru yang tidak tersedia dalam konteks.
- Jangan menjawab pertanyaan yang berada di luar topik perpajakan Indonesia, pelaporan pajak, atau akun pajak digital.
- Jawablah **dalam Bahasa Indonesia yang formal namun mudah dipahami**.
- Jika konteks tidak memuat informasi yang cukup, jawab dengan: `"Maaf, saya tidak memiliki informasi yang cukup untuk menjawab pertanyaan tersebut."`

**Pertanyaan dari pengguna:**
{question}

**Jawaban HARUS dalam Bahasa Indonesia:**
"""

prompt = PromptTemplate(
    template=template, 
    input_variables=[
        'context', 
        'question',
    ]
)

In [10]:
# Retrieval Chain
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3} # Mengambil 3 dokumen terdekat secara semantik
    ),
    chain_type_kwargs={"prompt": prompt}
)

## Test Query

### Test Query (10 Pertanyaan Seputar Pajak)

In [11]:
# Test Query (10 Pertanyaan Seputar Pajak)
questions = [
    "Apakah ada sanksi apabila saya tidak lapor pajak?",
    "Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?",
    "Apakah Wajib Pajak dapat diperiksa setelah melakukan permohonan pembetulan atau penghapusan?",
    "Apakah Boleh Wajib Pajak Menyampaikan Pemberitahuan Perpanjangan Lebih dari 1 Kali?",
    "Bagaimana cara masyarakat/pembeli mengecek e-Faktur merupakan Faktur Pajak yang valid?",
    "Dokumen apa saja yang harus disiapkan untuk lapor SPT Tahunan?",
    "Bagaimana tata cara pelaporan SPT Tahunan melalui eFiling?",
    "Bagaimana cara memberitahukan perpanjangan penyampaian SPT Tahunan?",
    "Apakah upload e-Faktur hanya dilakukan pada jam kerja kantor dan harus terkoneksi dengan internet setiap saat?",
    "Bagaimana cara mengisi detail data formulir yang digunakan untuk template SPT yang akan dibuat dan dilaporkan pada e-Form?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(f"Jawaban:\n{result['result']}")
    print(f"{'='*50}")


Pertanyaan: Apakah ada sanksi apabila saya tidak lapor pajak?


C:\Users\galuh\AppData\Local\Temp\ipykernel_15212\3866588145.py:18: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({"query": q})


Jawaban:
Apabila Anda tidak melaporkan atau terlambat melaporkan pajak, terdapat sanksi yang akan dikenakan berdasarkan Undang-undang Ketentuan Umum dan Tata Cara Perpajakan. Sanksi tersebut berupa denda administrasi dan sanksi pidana. Denda administrasi untuk SPT Tahunan Wajib Pajak Orang Pribadi adalah Rp. 100.000, sedangkan untuk Wajib Pajak Badan adalah Rp. 1.000.000. Sanksi pidana dapat berupa denda sebesar 100% sampai 400% dari pajak terhutang.

Pertanyaan: Dalam hal Wajib Pajak melakukan kesalahan berulang, bagaimana interpretasi khilaf, padahal diketahui bahwa Wajib Pajak sudah pernah diperiksa?
Jawaban:
Atas kesalahan yang dilakukan secara berulang, Wajib Pajak masih dinterpretasikan khilaf sepanjang memenuhi ketentuan dan persyaratan yang diatur dalam PMK-91/PMK.03/2015. Wajib Pajak tidak dapat dianggap khilaf atas suatu masa pajak atau tahun pajak apabila untuk masa pajak dan tahun pajak tersebut telah dilakukan pemeriksaan.

Pertanyaan: Apakah Wajib Pajak dapat diperiksa se

Insight: Chatbot berhasil menjawab semua pertanyaan yang berkaitan dengan Pajak sesuai dengan jawaban hasil scraping.

### Test Query (10 Pertanyaan Seputar Pajak Tanpa Keyword Perpajakan)

In [13]:
# Test Query (10 Pertanyaan Seputar Pajak Tanpa Keyword Perpajakan)
questions = [
    "Saya lupa password, bagaimana cara reset password",
    "Bagaimana cara buat akun?",
    "Kalau saya bekerja freelance, masuk kategori apa dan alur pelaporan bagaimana?",
    "Apakah saya boleh tidak lapor?",
    "Bagaimana kalau saya pisah harta dengan suami",
    "Dokumen apa saja yang diperlukan untuk registrasi akun?",
    "Kapan deadline pelaporan?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(f"Jawaban:\n{result['result']}")
    print(f"{'='*50}")


Pertanyaan: Saya lupa password, bagaimana cara reset password
Jawaban:
Apabila Anda lupa password, Anda dapat melakukan reset password melalui Akun PKP atau DJP Online. Untuk Akun PKP, Anda dapat membuka akun melalui web https://efaktur.pajak.go.id/login, kemudian klik link "lupa Password?" pada halaman login dan ikuti langkah-langkah yang disediakan. Sedangkan untuk DJP Online, Anda dapat masuk ke laman DJP Online, klik menu "lupa password?reset di sini?", dan ikuti instruksi yang diberikan. Jika Anda memiliki masalah dalam melakukan reset password, silakan hubungi pihak terkait.

Pertanyaan: Bagaimana cara buat akun?
Jawaban:
Untuk membuat akun, PKP yang memenuhi syarat akan dibuatkan Akun PKP oleh DJP. PKP harus mengaktifkan Akun tersebut dengan mengikuti langkah-langkah yang tertera, seperti mengisi identitas sesuai status perkawinan, mengisi kolom-kolom dengan benar, dan memasukkan kode keamanan sebelum mengklik "submit". Jika terdapat kolom baru "Langkah Berikutnya", PKP harus m

In [15]:
# Test Query (10 Pertanyaan Seputar Pajak Tanpa Keyword Perpajakan)
questions = [
    "Kapan batas waktu pelaporan?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(f"Jawaban:\n{result['result']}")
    print(f"{'='*50}")


Pertanyaan: Kapan batas waktu pelaporan?
Jawaban:
Batas waktu pelaporan pajak tidak diatur dalam PER-16/PJ/2014. Namun disarankan untuk melaporkan/upload e-Faktur sesegera mungkin untuk menghindari sanksi perpajakan yang berlaku.


Pertanyaan terakhir "Kapan deadline pelaporan" tidak dapat dijawab, padahal terkait perpajakan. Hal ini dikarenakan lingkup pertanyaannya yang luas / ambigu, dan juga dengan mix bahasa inggris. Jika pertanyaannya diganti menggunakan bahasa indonesia, bot berhasil menjawab meskipun tidak detail mengenai pelaporan pajak apa.

### Test Query (10 Pertanyaan ambigu diluar Pajak)

In [14]:
# Test Query (10 Pertanyaan ambigu diluar Pajak)
questions = [
    "Kapan terakhir kali saya lapor pajak?",
    "Siapa nama direktur kantor pajak sekarang ini?",
    "Berapa jumlah jendela di kantor pajak?",
    "kapan pajak pertama kali di temukan?",
    "Siapa teman baik Kodomo?",
    "Jam brapa kantor pajak buka?",
    "Berapa orang yang kerja di perpajakan?",
    "Apa itu Mobile legend?",
    "Kapan terakir kali kamu bersyukur?",
    "Ada berapa kantor pajak di Jakarta?"
]

for q in questions:
    print(f"\n{'='*50}")
    print(f"Pertanyaan: {q}")
    result = qa({"query": q})
    print(f"Jawaban:\n{result['result']}")
    print(f"{'='*50}")


Pertanyaan: Kapan terakhir kali saya lapor pajak?
Jawaban:
Maaf, saya tidak memiliki informasi yang cukup untuk menjawab pertanyaan tersebut.

Pertanyaan: Siapa nama direktur kantor pajak sekarang ini?
Jawaban:
Maaf, saya tidak memiliki informasi yang cukup untuk menjawab pertanyaan tersebut.

Pertanyaan: Berapa jumlah jendela di kantor pajak?
Jawaban:
Maaf, saya tidak memiliki informasi yang cukup untuk menjawab pertanyaan tersebut.

Pertanyaan: kapan pajak pertama kali di temukan?
Jawaban:
Maaf, saya tidak memiliki informasi yang cukup untuk menjawab pertanyaan tersebut.

Pertanyaan: Siapa teman baik Kodomo?
Jawaban:
Maaf, saya tidak memiliki informasi yang cukup untuk menjawab pertanyaan tersebut.

Pertanyaan: Jam brapa kantor pajak buka?
Jawaban:
Maaf, saya tidak memiliki informasi yang cukup untuk menjawab pertanyaan tersebut.

Pertanyaan: Berapa orang yang kerja di perpajakan?
Jawaban:
Maaf, saya tidak memiliki informasi yang cukup untuk menjawab pertanyaan tersebut.

Pertanyaan

Insight: Semua query non-contextual tidak dijawab oleh chatbot

## Conclusion

xxxx